<a href="https://colab.research.google.com/github/refercon/ipynb/blob/main/DWSP_MT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'bias-correction-ucl:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F558095%2F1015285%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240222%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240222T071839Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D58b1044f25a2afc6be7e47d7fd289a3e5c70e6e09390358596e3e81ba7d24b2ec0f019afb1f53a6ca201429041d723fef41597383b80215307f09aa5ec323baf7b91787230f41b4f3a85ee7cf5420d68c3727543de05ee1ca51139db69e43f8a9ee7aae508abffcc7583babe4d26132bbf2fcf88e4b24021fd6930011ec1b67bb2cb12129034d07f53dc9d6abd973138c698e9a91335be837470547a0ab93a2ec4a395dafb0f566fa0ddea725e1e1312e6c5af3a1869af25bb7ffb5edacab18780ead33a72d5af5bc0ff5584f7f50b8b9c1b644476618fbd2301a69b116e9950f205ea2ab0a93ca94c97e63f4e4f36884a9c77685b80e7a7ce5ad4b73437a0f9'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 654062 bytes downloaded
Downloaded and uncompressed: bias-correction-ucl
Data source import complete.


# 1. Dataset pre-processing

In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from functools import reduce
from sklearn import preprocessing
import warnings
warnings.filterwarnings('ignore')
df = pd.read_csv('../input/bias-correction-ucl/Bias_correction_ucl.csv')
#df['Worker'] = 'NULL'
df = df.reset_index(drop=True)
df = df.loc[:,['Date','LDAPS_Tmax_lapse','LDAPS_RHmax','LDAPS_CC2','LDAPS_WS','Solar radiation']]
df.columns = ['Location','TEMP','Humidity','Cloud','Wind-speed','SR']
data  = df.groupby('Location')
data_limit_10 = list(data)[0:20]
X = []
for i in range(len(data_limit_10)):
    location = 'L' + str(i + 1)
    data_limit_10[i][1]['Location'] = location
    X.append(data_limit_10)
X2 = []
for j in range(len(data_limit_10)):
    X1 = X[0][j][1].reset_index(drop=True)
    X1.columns = ['Location'+str(j+1),'TEMP'+str(j+1),
                  'Humidity'+str(j+1),'Cloud'+str(j+1),'Wind-speed'+str(j+1),'SR'+str(j+1)]
    X1.index = X1.index + 1
    X1.insert(0, 'Worker', 'GD-W' + X1.index.astype(str))
    X2.append(X1)
pd.set_option('max_columns',10)
pd.set_option('max_rows',10)
GD = reduce(lambda left,right: pd.merge(left,right,how='outer',on="Worker"), X2).fillna(method='pad')
GD['Label'] = 1
GD_TEMP = GD.filter(regex='Worker|TEMP')
GD_HUM = GD.filter(regex='Worker|Humidity')
GD_CLD = GD.filter(regex='Worker|Cloud')
GD_CLD.iloc[:,1:21] *= 100
GD_WSD = GD.filter(regex='Worker|Wind-speed')
GD_WSD.iloc[:,1:21] *=  2
GD_SR = GD.filter(regex='Worker|SR')
GD_SR.iloc[:,1:21] /= 100 # (GD_SR.iloc[:,1:21] - GD_SR.iloc[:,1:21].mean()) / GD_SR.iloc[:,1:21].std() #normalization
GD_TEMP.to_csv('gd.csv', index=False)

OptionError: Pattern matched multiple keys

# 2. Building unknown dataset

In [ ]:
# 80 workers("AT"， “TPA”, "Spammer")
# There are 10 time-steps in DPC, and DPC arranges 2 locations to workers in 1 time-step.
# 20 “TPA” satisfied accuracy 100% in a specific typle of task.
# 20 "AT" satisfied accuracy ？% in all tasks, but the accuracy is lower in a specific typle of task.
# 40 other workers satisfied distribution randomly.
# 20 “TPA” are generated by GD in a specific typle of task.For example,
# A “TPA” worker submits true data in a typle of TEMP and false data in a typle of humidity, and etc.
# 15 “TPA”  workers has a higher accuracy of single task.  5 workers has a higher accuracy of some tasks.
# 20 "AT" workers has a lower accuracy of single task , but has a higer accuracy of single location.
# 40 other workers: 30 workers satisfied accuracy 100% in a specific typle of task and not all of the time-steps.
# 40 other workers: 10 workers satisfied accuracy 0 in all of the time-steps.

# def font_color(val):
#     print(val)
#     color = 'green' if int(val) == 1 else 'red'
#     return 'color: %s' % color

# .sample(frac=1)
# UNK_workers_TEMP = X3.filter(regex='Worker|TEMP').drop(X3.index.to_list(),axis = 0)
TPA_TEMP = GD_TEMP.reset_index(drop=True).drop(GD_TEMP.tail(5).index).drop('Worker',axis=1)
TPA_HUM = GD_HUM.reset_index(drop=True).drop(GD_HUM.tail(5).index).drop('Worker',axis=1)
TPA_CLD = GD_CLD.reset_index(drop=True).drop(GD_CLD.tail(5).index).drop('Worker',axis=1)
TPA_WSD = GD_WSD.reset_index(drop=True).drop(GD_WSD.tail(5).index).drop('Worker',axis=1)
TPA_SR = GD_SR.reset_index(drop=True).drop(GD_SR.tail(5).index).drop('Worker',axis=1)

TPA_TEMP.index = TPA_TEMP.index + 1
TPA_TEMP.insert(0, 'Worker', 'UN-W' + TPA_TEMP.index.astype(str))
TPA_TEMP.insert(21, 'Label', 0)
TPA_TEMP.loc[0:3,'Label'] = 1
TPA_TEMP.loc[16:20,'Label'] = 1
TPA_TEMP.iloc[3:10,1:],TPA_TEMP.iloc[10:15,1:] = TPA_TEMP.iloc[3:10,1:] * 1.3,TPA_TEMP.iloc[10:15,1:] * 0.7

TPA_HUM.index = TPA_HUM.index + 1
TPA_HUM.insert(0, 'Worker', 'UN-W' + TPA_HUM.index.astype(str))
TPA_HUM.insert(21, 'Label', 0)
TPA_HUM.loc[4:6,'Label'] = 1
TPA_HUM.loc[16:20,'Label'] = 1
TPA_HUM.iloc[0:3,1:],TPA_HUM.iloc[6:15,1:] = TPA_HUM.iloc[0:3,1:] * 1.2,TPA_HUM.iloc[6:15,1:] * 0.8

TPA_CLD.index = TPA_CLD.index + 1
TPA_CLD.insert(0, 'Worker', 'UN-W' + TPA_CLD.index.astype(str))
TPA_CLD.insert(21, 'Label', 0)
TPA_CLD.loc[7:9,'Label'] = 1
TPA_CLD.loc[16:20,'Label'] = 1
TPA_CLD.iloc[0:6,1:],TPA_CLD.iloc[9:15,1:] = TPA_CLD.iloc[0:6,1:] / 3,TPA_CLD.iloc[9:15,1:] / 2

TPA_WSD.index = TPA_WSD.index + 1
TPA_WSD.insert(0, 'Worker', 'UN-W' + TPA_WSD.index.astype(str))
TPA_WSD.insert(21, 'Label', 0)
TPA_WSD.loc[10:12,'Label'] = 1
TPA_WSD.loc[16:20,'Label'] = 1
TPA_WSD.iloc[0:9,1:],TPA_WSD.iloc[12:15,1:] = TPA_WSD.iloc[0:9,1:] * 2,TPA_WSD.iloc[12:15,1:] / 2

TPA_SR.index = TPA_SR.index + 1
TPA_SR.insert(0, 'Worker', 'UN-W' + TPA_SR.index.astype(str))
TPA_SR.insert(21, 'Label', 0)
TPA_SR.loc[13:15,'Label'] = 1
TPA_SR.iloc[0:12,1:],TPA_SR.iloc[15:20,1:] = TPA_SR.iloc[0:12,1:] * 1.2 ,TPA_SR.iloc[15:20,1:] / 1.1

AT_TEMP = TPA_TEMP.append(GD_TEMP.iloc[:20,1:])
AT_TEMP = AT_TEMP.reset_index(drop=True)
AT_TEMP.index = AT_TEMP.index + 1
AT_TEMP.loc[21:40,'Worker'] = 'UN-W' + AT_TEMP.loc[21:40,'Worker'].index.astype(str)
AT_TEMP.loc[21:40,'Label'] = 2
AT_TEMP.iloc[20:40,1:6] = AT_TEMP.iloc[20:40,1:6] * 1.3

AT_HUM = TPA_HUM.append(GD_HUM.iloc[:20,1:])
AT_HUM = AT_HUM.reset_index(drop=True)
AT_HUM.index = AT_HUM.index + 1
AT_HUM.loc[21:40,'Worker'] = 'UN-W' + AT_TEMP.loc[21:40,'Worker'].index.astype(str)
AT_HUM.loc[21:40,'Label'] = 2
AT_HUM.iloc[20:40,1:6],AT_HUM.iloc[20:40,19:21] = AT_HUM.iloc[20:40,1:6] * 1.2,AT_HUM.iloc[20:40,1:6] * .8

AT_CLD = TPA_CLD.append(GD_CLD.iloc[:20,1:])
AT_CLD = AT_CLD.reset_index(drop=True)
AT_CLD.index = AT_CLD.index + 1
AT_CLD.loc[21:40,'Worker'] = 'UN-W' + AT_CLD.loc[21:40,'Worker'].index.astype(str)
AT_CLD.loc[21:40,'Label'] = 2
AT_CLD.iloc[20:40,1:6] = AT_CLD.iloc[20:40,1:6] / 2

AT_WSD = TPA_WSD.append(GD_WSD.iloc[:20,1:])
AT_WSD = AT_WSD.reset_index(drop=True)
AT_WSD.index = AT_WSD.index + 1
AT_WSD.loc[21:40,'Worker'] = 'UN-W' + AT_WSD.loc[21:40,'Worker'].index.astype(str)
AT_WSD.loc[21:40,'Label'] = 2
AT_WSD.iloc[20:40,1:6],AT_WSD.iloc[20:40,19:21] = AT_WSD.iloc[20:40,1:6] * 2,AT_WSD.iloc[20:40,1:6] / 2

AT_SR = TPA_SR.append(GD_SR.iloc[:20,1:])
AT_SR = AT_SR.reset_index(drop=True)
AT_SR.index = AT_SR.index + 1
AT_SR.loc[21:40,'Worker'] = 'UN-W' + AT_SR.loc[21:40,'Worker'].index.astype(str)
AT_SR.loc[21:40,'Label'] = 2
AT_SR.iloc[20:40,1:6],AT_SR.iloc[20:40,19:21] = AT_SR.iloc[20:40,1:6] * 2,AT_SR.iloc[20:40,1:6] / 2

OW_TEMP = AT_TEMP.append(AT_TEMP,ignore_index=True)
OW_TEMP.loc[40:,'Worker'] = 'UN-W' + (OW_TEMP.loc[40:,'Worker'].index +1).astype(str)
OW_TEMP.loc[40:,'Label'] = 0
OW_TEMP.iloc[40:70,1:11] = OW_TEMP.iloc[40:70,1:11] * 1.3
OW_TEMP.iloc[70:80,1:21] = OW_TEMP.iloc[70:80,1:21] * 0.7

OW_HUM = AT_HUM.append(AT_HUM,ignore_index=True)
OW_HUM.loc[40:,'Worker'] = 'UN-W' + (OW_HUM.loc[40:,'Worker'].index +1).astype(str)
OW_HUM.loc[40:,'Label'] = 0
OW_HUM.iloc[40:70,1:11] = OW_HUM.iloc[40:70,1:11] * 1.3
OW_HUM.iloc[70:80,1:21] = OW_HUM.iloc[70:80,1:21] * 0.7

OW_CLD = AT_CLD.append(AT_CLD,ignore_index=True)
OW_CLD.loc[40:,'Worker'] = 'UN-W' + (OW_CLD.loc[40:,'Worker'].index +1).astype(str)
OW_CLD.loc[40:,'Label'] = 0
OW_CLD.iloc[40:70,10:21] = OW_CLD.iloc[40:70,10:21] / 3
OW_CLD.iloc[70:80,1:21] = OW_CLD.iloc[70:80,1:21] / 2

OW_WSD = AT_WSD.append(AT_WSD,ignore_index=True)
OW_WSD.loc[40:,'Worker'] = 'UN-W' + (OW_WSD.loc[40:,'Worker'].index +1).astype(str)
OW_WSD.loc[40:,'Label'] = 0
OW_WSD.iloc[40:70,1:11] = OW_WSD.iloc[40:70,1:11] * 2
OW_WSD.iloc[70:80,1:21] = OW_WSD.iloc[70:80,1:21] / 2

OW_SR = AT_SR.append(AT_SR,ignore_index=True)
OW_SR.loc[40:,'Worker'] = 'UN-W' + (OW_SR.loc[40:,'Worker'].index +1).astype(str)
OW_SR.loc[40:,'Label'] = 0
OW_SR.iloc[40:70,10:21] = OW_SR.iloc[40:70,10:21] * 2
OW_SR.iloc[70:80,1:21] = OW_SR.iloc[70:80,1:21] / 2

pd.set_option('max_columns',10)
pd.set_option('max_rows',100)
OW_TEMP
OW_HUM
OW_CLD
OW_WSD
OW_SR

# **3. Data classfication**

In [ ]:
from sklearn import svm
from sklearn.metrics import accuracy_score,f1_score
from sklearn.model_selection import GridSearchCV
import warnings
from IPython.display import display
warnings.filterwarnings('ignore')
GD_TEMP_fit = GD_TEMP.drop('Worker',axis=1)
UN_TEMP_fit = OW_TEMP.drop(['Worker','Label'],axis=1)

GD_HUM_fit = GD_HUM.drop('Worker',axis=1)
UN_HUM_fit = OW_HUM.drop(['Worker','Label'],axis=1)

GD_CLD_fit = GD_CLD.drop('Worker',axis=1)
UN_CLD_fit = OW_CLD.drop(['Worker','Label'],axis=1)

GD_WSD_fit = GD_WSD.drop('Worker',axis=1)
UN_WSD_fit = OW_WSD.drop(['Worker','Label'],axis=1)

GD_SR_fit = GD_SR.drop('Worker',axis=1)
UN_SR_fit = OW_SR.drop(['Worker','Label'],axis=1)

pd.set_option('max_columns',20)
pd.set_option('max_rows',100)

# nus = [0.1,0.01, 0.02, 0.1, 1]
# gammas = [0.1,0.01, 0.02, 0.1, 1]
# tuned_parameters = {'kernel' : ['rbf'], 'gamma' : gammas, 'nu': nus}
# scores = ['accuracy', 'f1-score']
# for score in scores:
#     estimator = GridSearchCV(svm.OneClassSVM(), tuned_parameters, cv=10,
#                            scoring='f1' , return_train_score=True)

#     estimator.fit(GD_TEMP_fit.iloc[0:19,0:2],GD.iloc[0:19,-1])
#     y_predict = estimator.predict(UN_TEMP_fit.iloc[0:19,0:2])
# #     print(y_predict)
#     resultDf = pd.DataFrame(estimator.cv_results_)
# #     print(resultDf[["mean_test_score", "std_test_score", "params"]].sort_values(by=["mean_test_score"],
# #                                                                                 ascending=False).head())
#     print(estimator.best_params_)
# resultDf
search_params = pd.DataFrame({'nus':[0],'GD_ACC':[0],'GD_F1score':[0],'Test_ACC':[0],'Test_F1score':[0]})
nus = 0
i = 0
while nus <= 0.99:
    nus += 0.01
    i += 1
    estimator= svm.OneClassSVM(nu=nus, kernel="rbf", gamma=0.01)
#     for t in range(20):
#         if t % 2 == 0:
    GD_temp_shuffle = GD_TEMP_fit.iloc[:,0:2].sample(frac=1)
    UN_temp_shuffle = UN_TEMP_fit.iloc[:,0:2]
    estimator.fit(GD_temp_shuffle)
    y_pred_temp_gd = estimator.predict(GD_temp_shuffle)
    y_pred_temp = estimator.predict(UN_temp_shuffle)
#     TEMP = OW_TEMP.loc[:,['Worker','TEMP'+str(t + 1),'TEMP'+str(t + 2)]]
#     predict_TEMP.append(TEMP[y_pred_temp==1])
    for j in range(len(OW_TEMP.loc[0:19,'Label'])):
        if OW_TEMP.loc[0:19,'Label'][j] != 1:
            OW_TEMP.loc[0:19,'Label'][j] = -1
    search_params.loc[i] = [nus,accuracy_score(GD.loc[0:19,'Label'],y_pred_temp_gd[0:20]),
                            f1_score(GD.loc[0:19,'Label'],y_pred_temp_gd[0:20]),
                            accuracy_score(OW_TEMP.loc[0:19,'Label'],y_pred_temp[0:20]),
                            f1_score(OW_TEMP.loc[0:19,'Label'],y_pred_temp[0:20])]
search_params.drop([0])

In [ ]:
import time
start_time = time.time()
end_time = time.time()
##### from sklearn import svm
from sklearn.metrics import accuracy_score,f1_score
import warnings
import matplotlib.pyplot  as plt
from IPython.display import display
warnings.filterwarnings('ignore')
vars()['GT_fit'] = [GD_TEMP.drop('Worker',axis=1),GD_HUM.drop('Worker',axis=1),GD_CLD.drop('Worker',axis=1),GD_WSD.drop('Worker',axis=1),
                    GD_SR.drop('Worker',axis=1)]
vars()['UN_fit'] = [OW_TEMP.drop(['Worker','Label'],axis=1),OW_HUM.drop(['Worker','Label'],axis=1),
                    OW_CLD.drop(['Worker','Label'],axis=1),OW_WSD.drop(['Worker','Label'],axis=1),
                    OW_SR.drop(['Worker','Label'],axis=1)]
vars()['reliable'] = [[],[],[],[],[]]
vars()['unreliable'] = [[],[],[],[],[]]
task = ['TEMP','Humidity','Cloud','Wind-speed','SR']
vars()['ACC'] = [pd.DataFrame({'Task':[0],'Cycle':[0],'ACC':[0],'F1-score':[0]}),
                 pd.DataFrame({'Task':[0],'Cycle':[0],'ACC':[0],'F1-score':[0]}),
                 pd.DataFrame({'Task':[0],'Cycle':[0],'ACC':[0],'F1-score':[0]}),
                 pd.DataFrame({'Task':[0],'Cycle':[0],'ACC':[0],'F1-score':[0]}),
                 pd.DataFrame({'Task':[0],'Cycle':[0],'ACC':[0],'F1-score':[0]})]

vars()['OW'] = [OW_TEMP,OW_HUM,OW_CLD,OW_WSD,OW_SR]
estimator= svm.OneClassSVM(nu=0.01, kernel="rbf", gamma=0.01)
for f in range(len(vars()['reliable'])):
    for t in range(20):
        if t % 2 == 0:
            estimator.fit(GT_fit[f].iloc[:,t:t+2])
            y_pred_temp = estimator.predict(UN_fit[f].iloc[:,t:t + 2])
            reliable[f].append(OW[f].loc[:,['Worker',task[f]+str(t + 1),task[f]+str(t + 2)]][y_pred_temp==1])
            unreliable[f].append(OW[f].loc[:,['Worker',task[f]+str(t + 1),task[f]+str(t + 2)]][y_pred_temp==-1])#-1 unlabelled..
            for i in range(len(OW[f].loc[0:19,'Label'])):
                if OW[f].loc[0:19,'Label'][i] != 1:
                    OW[f].loc[0:19,'Label'][i] = -1
            ACC[f].loc[int(t/2)] = [task[f],'t'+str(int(t/2 + 1)),accuracy_score(OW[f].loc[0:19,'Label'],y_pred_temp[0:20]),
                                    f1_score(OW[f].loc[0:19,'Label'],y_pred_temp[0:20])]

execution_time = end_time - start_time
print("程序执行时间：", execution_time, "秒")


pd.set_option('max_columns',20)
pd.set_option('max_rows',20)

ACC[0].merge(ACC[1].merge(ACC[2].merge(ACC[3].merge(ACC[4],on="Cycle"),on="Cycle"),on="Cycle"),on="Cycle")

# unreliable_all = []
# for i in range(len(vars()['unreliable'])):
#     for j in range(10):
#         unreliable_all.append(unreliable[i][j])

# unreliable_all = reduce(lambda left,right: pd.merge(left,right,how='outer',on="Worker"),unreliable_all)
# unreliable_all

#分类效果图

In [ ]:
#Visualize the process of training and predicting.
plt.figure(figsize = (30,4))
size = 14
plt.subplot(151) #Temperature
time_step1 = 0
time_step2 = 2
lam=0.01
estimator= svm.OneClassSVM(nu=lam, kernel="rbf", gamma=0.01)

estimator.fit(GT_fit[0].iloc[:,time_step1:time_step2])
predict_0 = estimator.predict(UN_fit[0].iloc[:,time_step1:time_step2])
reliable_pre = UN_fit[0].iloc[:,time_step1:time_step2][predict_0==1]
unreliable_pre = UN_fit[0].iloc[:,time_step1:time_step2][predict_0==-1]

xx, yy = np.meshgrid(np.linspace(0, 70, 50), np.linspace(0, 70, 50))
Z = estimator.decision_function(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)
plt.contourf(xx, yy, Z, cmap=plt.cm.Blues)
plt.contourf(xx, yy, Z, levels=np.linspace(Z.min(), 0, 10), cmap=plt.cm.Blues)
plt.contour(xx, yy, Z, levels=[0], linewidths=1, colors='red')
plt.contourf(xx, yy, Z, levels=[0, Z.max()], colors='palevioletred')

a = plt.scatter(GT_fit[0].iloc[:,time_step1:time_step1+1], GT_fit[0].iloc[:,time_step1+1:time_step2], c='white', s=60, edgecolors='k', marker='o')
b = plt.scatter(reliable_pre.iloc[:,0:1], reliable_pre.iloc[:,1:2], c='#D00000', s=60,edgecolors='k', marker='o')
c = plt.scatter(unreliable_pre.iloc[:,0:1], unreliable_pre.iloc[:,1:2],  c='#FFBA08', edgecolor='k', s=60, marker='x')
plt.axis('tight')
plt.xlim((10, 55))
plt.ylim((10, 55))
plt.xticks(size=size)
plt.yticks(size=size)
plt.text(15,50, f'$\lambda={lam}$',fontsize=size)
plt.legend([a, b, c],[ "Ground truth","Reliable data", "Unreliable data"],loc="lower right", prop={'size': size})
# plt.xlabel('X1',fontsize=15)
# plt.ylabel('X2',fontsize=15)
# plt.grid(linestyle=":")

plt.subplot(152)#Humidity
estimator.fit(GT_fit[1].iloc[:,time_step1:time_step2])
predict_1 = estimator.predict(UN_fit[1].iloc[:,time_step1:time_step2])
reliable_pre = UN_fit[1].iloc[:,time_step1:time_step2][predict_1==1]
unreliable_pre = UN_fit[1].iloc[:,time_step1:time_step2][predict_1==-1]
xx, yy = np.meshgrid(np.linspace(0, 160, 50), np.linspace(0, 160, 50))
Z = estimator.decision_function(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)
plt.contourf(xx, yy, Z, cmap=plt.cm.Blues)
plt.contourf(xx, yy, Z, levels=np.linspace(Z.min(), 0, 15), cmap=plt.cm.Blues)
plt.contour(xx, yy, Z, levels=[0], linewidths=1, colors='red')
plt.contourf(xx, yy, Z, levels=[0, Z.max()], colors='palevioletred')

a = plt.scatter(GT_fit[1].iloc[:,time_step1:time_step1+1], GT_fit[1].iloc[:,time_step1+1:time_step2], c='white', s=60, edgecolors='k', marker='o')
b = plt.scatter(reliable_pre.iloc[:,0:1], reliable_pre.iloc[:,1:2], c='#D00000', s=60,edgecolors='k', marker='o')
c = plt.scatter(unreliable_pre.iloc[:,0:1], unreliable_pre.iloc[:,1:2],  c='#FFBA08', edgecolor='k', s=60, marker='x')
plt.axis('tight')
plt.xlim((50, 160))
plt.ylim((50, 160))
plt.xticks(size=size)
plt.yticks(size=size)
plt.text(65,145, f'$\lambda={lam}$',fontsize=size)
plt.legend([a, b, c],[ "Ground truth","Reliable data", "Unreliable data"],loc="lower right", prop={'size': size})
# plt.xlabel('X1',fontsize=15)
# plt.ylabel('X2',fontsize=15)
# plt.grid(linestyle=":")

plt.subplot(153)#Cloud
estimator.fit(GT_fit[2].iloc[:,time_step1:time_step2])
predict_2 = estimator.predict(UN_fit[2].iloc[:,time_step1:time_step2])
reliable_pre = UN_fit[2].iloc[:,time_step1:time_step2][predict_2==1]
unreliable_pre = UN_fit[2].iloc[:,time_step1:time_step2][predict_2==-1]
xx, yy = np.meshgrid(np.linspace(0, 120, 50), np.linspace(0, 120, 50))
Z = estimator.decision_function(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)
plt.contourf(xx, yy, Z, cmap=plt.cm.Blues)
plt.contourf(xx, yy, Z, levels=np.linspace(Z.min(), 0, 15), cmap=plt.cm.Blues)
plt.contour(xx, yy, Z, levels=[0], linewidths=1, colors='red')
plt.contourf(xx, yy, Z, levels=[0, Z.max()], colors='palevioletred')
a = plt.scatter(GT_fit[2].iloc[:,time_step1:time_step1+1], GT_fit[2].iloc[:,time_step1+1:time_step2], c='white', s=60, edgecolors='k', marker='o')
b = plt.scatter(reliable_pre.iloc[:,0:1], reliable_pre.iloc[:,1:2], c='#D00000', s=60,edgecolors='k', marker='o')
c = plt.scatter(unreliable_pre.iloc[:,0:1], unreliable_pre.iloc[:,1:2],  c='#FFBA08', edgecolor='k', s=60, marker='x')
plt.axis('tight')
plt.text(5,105, f'$\lambda={lam}$',fontsize=size)
plt.xlim((0, 60))
plt.ylim((10, 120))
plt.xticks(size=size)
plt.yticks(size=size)
plt.legend([a, b, c],[ "Ground truth","Reliable data", "Unreliable data"],loc="lower right", prop={'size': size})
# plt.xlabel('X1',fontsize=15)
# plt.ylabel('X2',fontsize=15)
# plt.grid(linestyle=":")

plt.subplot(154)#Wind-speed
estimator.fit(GT_fit[3].iloc[:,time_step1:time_step2])
predict_3 = estimator.predict(UN_fit[3].iloc[:,time_step1:time_step2])
reliable_pre = UN_fit[3].iloc[:,time_step1:time_step2][predict_3==1]
unreliable_pre = UN_fit[3].iloc[:,time_step1:time_step2][predict_3==-1]
xx, yy = np.meshgrid(np.linspace(-5, 120, 50), np.linspace(-5, 120, 50))
Z = estimator.decision_function(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)
plt.contourf(xx, yy, Z, cmap=plt.cm.Blues)
plt.contourf(xx, yy, Z, levels=np.linspace(Z.min(), 0, 15), cmap=plt.cm.Blues)
plt.contour(xx, yy, Z, levels=[0], linewidths=1, colors='red')
plt.contourf(xx, yy, Z, levels=[0, Z.max()], colors='palevioletred')
a = plt.scatter(GT_fit[3].iloc[:,time_step1:time_step1+1], GT_fit[3].iloc[:,time_step1+1:time_step2], c='white', s=60, edgecolors='k', marker='o')
b = plt.scatter(reliable_pre.iloc[:,0:1], reliable_pre.iloc[:,1:2], c='#D00000', s=60,edgecolors='k', marker='o')
c = plt.scatter(unreliable_pre.iloc[:,0:1], unreliable_pre.iloc[:,1:2],  c='#FFBA08', edgecolor='k', s=60, marker='x')
plt.text(5,105,  f'$\lambda={lam}$',fontsize=14)
plt.axis('tight')
plt.xlim((-5, 60))
plt.ylim((-5, 120))
plt.xticks(size=size)
plt.yticks(size=size)
plt.legend([a, b, c],[ "Ground truth","Reliable data", "Unreliable data"],loc="lower right", prop={'size': size})
# plt.xlabel('X1',fontsize=15)
# plt.ylabel('X2',fontsize=15)
# plt.grid(linestyle=":")

plt.subplot(155)#Solar-radiation
estimator.fit(GT_fit[4].iloc[:,time_step1:time_step2])
predict_4 = estimator.predict(UN_fit[4].iloc[:,0:2])
reliable_pre = UN_fit[4].iloc[:,time_step1:time_step2][predict_4==1]
unreliable_pre = UN_fit[4].iloc[:,time_step1:time_step2][predict_4==-1]
xx, yy = np.meshgrid(np.linspace(-5, 80, 50), np.linspace(-5, 80, 50))
Z = estimator.decision_function(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)
plt.contourf(xx, yy, Z, cmap=plt.cm.Blues)
plt.contourf(xx, yy, Z, levels=np.linspace(Z.min(), 0, 10), cmap=plt.cm.Blues)
plt.contour(xx, yy, Z, levels=[0], linewidths=1, colors='red')
plt.contourf(xx, yy, Z, levels=[0, Z.max()], colors='palevioletred')
a = plt.scatter(GT_fit[4].iloc[:,time_step1:time_step1+1], GT_fit[4].iloc[:,time_step1+1:time_step2], c='white', s=60, edgecolors='k', marker='o')
b = plt.scatter(reliable_pre.iloc[:,0:1], reliable_pre.iloc[:,1:2], c='#D00000', s=60,edgecolors='k', marker='o')
c = plt.scatter(unreliable_pre.iloc[:,0:1], unreliable_pre.iloc[:,1:2], c='#FFBA08',edgecolor='k', s=60, marker='x')
plt.text(45,75, f'$\lambda={lam}$',fontsize=size)
plt.axis('tight')
plt.xlim((40, 80))
plt.ylim((40, 80))
plt.xticks(size=size)
plt.yticks(size=size)
plt.legend([a, b, c],[ "Ground truth","Reliable data", "Unreliable data"],loc="lower right", prop={'size': size})
# plt.xlabel('X1',fontsize=15)
# plt.ylabel('X2',fontsize=15)
# plt.grid(linestyle=":")
plt.savefig(f'{lam}.svg',dpi=600,bbox_inches = 'tight')
plt.show()
GT_fit[0]

In [ ]:
font1 = {
'weight' : 'normal',
'size' : 20,
}
fontsize = 20
size = 14
plt.figure(figsize = (30,10))
plt.subplots_adjust(left=None,bottom=None,right=None,top=None,wspace=0.3)
plt.subplot(251)
original_data = plt.scatter(UN_fit[0].iloc[:,0:1],UN_fit[0].iloc[:,1:2],marker=".",facecolor='#699BCD',s=120)
abnomal_data = plt.scatter(UN_fit[0].iloc[:,0:1][predict_0==-1],UN_fit[0].iloc[:,1:2][predict_0==-1],
                           marker="o",facecolor='none',edgecolors='orange',s=80)
plt.legend((abnomal_data,original_data),('Unreliable data',
                                         'Reliable data'),prop=font1)
plt.grid(linestyle=':')
plt.tick_params(labelsize=fontsize)

plt.subplot(252)
original_data = plt.scatter(UN_fit[1].iloc[:,0:1],UN_fit[1].iloc[:,1:2],marker=".",facecolor='#699BCD',s=120)
abnomal_data = plt.scatter(UN_fit[1].iloc[:,0:1][predict_1==-1],UN_fit[1].iloc[:,1:2][predict_1==-1],
                           marker="o",facecolor='none',edgecolors='orange',s=80)
plt.legend((abnomal_data,original_data),('Unreliable data',
                                         'Reliable data'),prop=font1)
plt.grid(linestyle=':')
plt.tick_params(labelsize=fontsize)

plt.subplot(253)
original_data = plt.scatter(UN_fit[2].iloc[:,0:1],UN_fit[2].iloc[:,1:2],marker=".",facecolor='#699BCD',s=120)
abnomal_data = plt.scatter(UN_fit[2].iloc[:,0:1][predict_2==-1],UN_fit[2].iloc[:,1:2][predict_2==-1],
                           marker="o",facecolor='none',edgecolors='orange',s=80)
plt.legend((abnomal_data,original_data),('Unreliable data',
                                         'Reliable data'),prop=font1,loc="lower right")
plt.grid(linestyle=':')
plt.tick_params(labelsize=fontsize)

plt.subplot(254)#Postive data distribution
original_data = plt.scatter(UN_fit[3].iloc[:,0:1],UN_fit[3].iloc[:,1:2],marker=".",facecolor='#699BCD',s=120)
abnomal_data = plt.scatter(UN_fit[3].iloc[:,0:1][predict_3==-1],UN_fit[3].iloc[:,1:2][predict_3==-1],
                           marker="o",facecolor='none',edgecolors='orange',s=80)

plt.legend((abnomal_data,original_data),('Unreliable data',
                                         'Reliable data'),prop=font1,loc="upper left")
plt.grid(linestyle=':')
plt.tick_params(labelsize=fontsize)

plt.subplot(255)#Postive data distribution
original_data = plt.scatter(UN_fit[4].iloc[:,0:1],UN_fit[4].iloc[:,1:2],marker=".",facecolor='#699BCD',s=120)
abnomal_data = plt.scatter(UN_fit[4].iloc[:,0:1][predict_4==-1],UN_fit[4].iloc[:,1:2][predict_4==-1],
                           marker="o",facecolor='none',edgecolors='orange',s=80)

plt.legend((abnomal_data,original_data),('Unreliable data',
                                         'Reliable data'),prop=font1)
plt.grid(linestyle=':')
plt.tick_params(labelsize=fontsize)

plt.savefig('1-1.svg',dpi=600,bbox_inches = 'tight')
plt.show()


In [ ]:
import sklearn as skl
import matplotlib.pyplot as plt
from cycler import cycler
plt.figure(figsize = (30,4))
plt.rc('axes', prop_cycle=(cycler('color', ['#F51420','#C06000','#777E00','#008B00', '#008EA0']) +
                           cycler('linestyle', ['-', '--', ':', '-.', '-.'])))

def plot_roc(t,y_test, y_pred,subsize,label_postion="lower right"):
    fpr, tpr, thresholds = skl.metrics.roc_curve(y_test, y_pred, pos_label=1)
    print(fpr,',', tpr)

    roc_auc = skl.metrics.auc(fpr, tpr)
    lw = 2
    plt.subplot(1,5,subsize)
    plt.plot(fpr, tpr, lw=lw, label=t+' ROC (AUC={0:.2f})'.format(roc_auc))

    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False positive rate')
    plt.ylabel('True positive rate')
    plt.legend(loc=label_postion)
    plt.grid(linestyle=":")

plot_roc('TEMP',OW[0].loc[0:19,'Label'], predict_0[0:20],1)
plot_roc('HUM',OW[1].loc[0:19,'Label'], predict_1[0:20],1)
plot_roc('CLD',OW[2].loc[0:19,'Label'], predict_2[0:20],1)
plot_roc('WSD',OW[3].loc[0:19,'Label'], predict_3[0:20],1)
plot_roc('SR',OW[4].loc[0:19,'Label'], predict_4[0:20],1)
plt.savefig('2.svg',dpi=600,bbox_inches = 'tight')
plt.show()

# 4. Calculate trust according to the reliable data

In [ ]:
import time
start_time = time.time()
end_time = time.time()
pd.set_option('max_columns',10)
pd.set_option('max_rows',10)
vars()['reliable'] = [reliable[0],reliable[1],reliable[2],reliable[3],reliable[4]]
task = ['TEMP','Humidity','Cloud','Wind-speed','SR']
for i in range(len(vars()['reliable'])):
    vars()['reliable_trust_'+str(i)] = reduce(lambda left,right: pd.merge(left,right,how='outer',on="Worker"), reliable[i]).fillna(0)
    vars()['reliable_trust_'+str(i)]['Correct'] = 0.00
    for j in range(len( vars()['reliable_trust_'+str(i)])):
        m = 0
        for k in range(20):
            if  vars()['reliable_trust_'+str(i)][task[i]+str(k+1)][j] > 0:
                m = m + 1
        vars()['reliable_trust_'+str(i)]['Correct'][j] = m / 20
execution_time = end_time - start_time
print("程序执行时间：", execution_time, "秒")
reliable_trust_0

In [ ]:
import time
start_time = time.time()
end_time = time.time()

t = 0
a = 2
b = 1
vars()['trust_task'] = [[],[],[],[],[]]
vars()['trust_ascend_task'] = [[],[],[],[],[]]
for k in range(10):
    k +=  1
    t += (10 - k)

for f in range(len(reliable)):
    for i in range(len(vars()['reliable_trust_'+str(f)])):
        trust_score = 0
        init_trust = 0.5
        trust_ascend = 0
        t_cache = []
        trust_ascend_data_t = []
        for j in range(20):
            if j % 2 == 0:
                p = (j / 2 ) / t
#                 print(p)
                if (0 < vars()['reliable_trust_'+str(f)][task[f]+str(j+1)][i]) and (0 < vars()['reliable_trust_'+str(f)][task[f]+str(j+2)][i]):
                    init_trust +=  (b - init_trust) / a
                else:
                    trust_score += 1
                    init_trust -= (b * init_trust) / a
                t_cache.append(init_trust * p)
                trust_ascend += init_trust * p
                trust_ascend_data_ =  vars()['reliable_trust_'+str(f)]['Worker'][i],'t'+str(int(j/2+1)),trust_ascend
                trust_ascend_data_t.append(trust_ascend_data_)
        trust_ascend_task[f].append(trust_ascend_data_t)

        trust_data_ = vars()['reliable_trust_'+str(f)]['Worker'][i],sum(t_cache)
        trust_task[f].append(trust_data_)
    trus_ascend_modify = []
    for m in range(len(trust_ascend_task[f])):
        work_tust_ascend = pd.DataFrame(trust_ascend_task[f][m]) #worker1
        work_tust_ascend.columns = ['Worker','Cycle','Current_trust']
        trus_ascend_modify.append(work_tust_ascend)
    vars()['trust_all_'+str(f)]= reduce(lambda left,right: pd.merge(left,right,how='outer',on="Cycle"), trus_ascend_modify)
    pd.set_option('display.max_columns',100)
    pd.set_option('display.max_rows',20)
execution_time = end_time - start_time
print("程序执行时间：", execution_time, "秒")
trust_all_0 #TMEP relability  #t1 = 2 location data

# 5. Discover "TPA"

In [ ]:
import time
start_time = time.time()
end_time = time.time()
pd.set_option('display.max_rows',200)
pd.set_option('display.max_columns',100)
threshold_TPA = 0.9
for i in range(len(vars()['reliable'])):
    trust_all = pd.DataFrame(trust_task[i])
    trust_all.columns = ['Worker','Trust']
    trust_all.insert(0,'Task',task[i])

#     print(trust_all['Trust'])
    vars()['TPA_'+str(i)] = trust_all[trust_all['Trust'] >= threshold_TPA].reset_index()
    vars()['UNK_'+str(i)]  = trust_all[trust_all['Trust'] < threshold_TPA].reset_index()
# unlabelled_worker.sort_values('Data_trust',ascending=True).reset_index()
# spammer.sort_values('Data_trust',ascending=True).reset_index()
T = TPA_0.merge(TPA_1.merge(TPA_2.merge(TPA_3.merge(TPA_4,how="outer",on="Worker"),how="outer",on="Worker"),
                            how="outer",on="Worker"),how="outer",on="Worker")
TPA = T.filter(regex='Worker|Task|Trust').reset_index()
TPA
#R^{threshold} 首先我们已经计算出了所有工人在任务中可靠度R，R的最高值为1，越接近1表明，工人具有在任务中的可靠度越高
#因此，那些等于1的可靠度的工人，我们将其归类为TPAs,也就是阈值R^{threshold}=1。
#但实际环境中，可能存在所有的工人的可靠度R都无法达到1。
#为了应对这种情况，我们对R^{threshold}设置一个其他值 使其靠近1的状态，具体的计算可根据数据可视化、专家决策，排序法等选择最优的阈值，例如0.9等。排序取出前10然后均值、
#若工人在参与的所有的任务中的可靠度R小于R^{threshold}，并且在每个任务中的可靠度R大于0.5，也就是表明他们有完成所有的任务50%的概率，我们将其归类为ARs。
#对于剩余部分的工人，可根据实际情况重新检测或判定为spammer.
#在我们的实验中，我们根据数据观察，发现工人在相关任务的最高可靠度R是0.989008，有14个工人具有这样的可靠度，且满足任务分配需要，因此我们就设置R^{threshold}>=0.989008.

# 6. Discover "AT"

In [ ]:
# pd.set_option('display.max_rows',10)
# pd.set_option('display.max_columns',10)
# vars()['UN_fit'] = [OW_TEMP.drop(['Label'],axis=1),OW_HUM.drop(['Label'],axis=1),
#                     OW_CLD.drop(['Label'],axis=1),OW_WSD.drop(['Label'],axis=1),
#                     OW_SR.drop(['Label'],axis=1)]
# UNK = []
# for i in range(len(vars()['UN_fit'])):
#     UNK.append(UN_fit[i])
# UNK_all = reduce(lambda left,right: pd.merge(left,right,how='outer',on="Worker"),UNK)
# UNK_data = pd.concat([UNK_all.filter(regex='Worker'), T.filter(regex='Worker')]).drop_duplicates(keep=False).reset_index()
# UNK_data.merge(UNK_all,how='left',on="Worker")
threshold_AT = 0.6
T = UNK_0.merge(UNK_1.merge(UNK_2.merge(UNK_3.merge(UNK_4,how="outer",on="Worker"),how="outer",on="Worker"),
                            how="outer",on="Worker"),how="outer",on="Worker")
UNK = T.filter(regex='Worker|Task|Trust')
UNK_all = UNK[~UNK['Worker'].isin(TPA['Worker'])].filter(regex='Worker|Trust').reset_index().fillna(0)   #每个task  50%正确率。为全能型人才  根据信任度反推  个数。。。。。
UNK_all['T'] = UNK_all.iloc[:,2:].sum(axis=1) / 5
AT = UNK_all[UNK_all['T'] >= threshold_AT]
AT.columns = ['Index','Worker','TEMP_T','HUM_T','Cloud_T','Wind-speed_T','SR_T','C_T']
AT.reset_index()

# 7. Discovery "UNK"

In [ ]:
threshold_UNK = 0.1
UNK_all[(UNK_all['T'] < threshold_AT) & (UNK_all['T'] >= threshold_UNK)]

# 8. Discovery "Spammer"

In [ ]:
UNK_all[UNK_all['T'] < threshold_UNK] #There need to add the worker whose T is 0 ....

# 9. Evaluate Data Bias of Our Schema

In [ ]:
# A = OW_HUM[OW_HUM['Worker'].isin(TPA_1['Worker'])].filter(regex='Hum').reset_index().drop('index',axis=1)
# B = GD_HUM.loc[:5,].filter(regex='Hum')
# abs(B - A) / B

D = []
E_all = [ pd.DataFrame({'Bias_TEMP':[0]}) ,pd.DataFrame({'Bias_Humidity':[0]}) ,pd.DataFrame({'Bias_cloud':[0]}) ,
         pd.DataFrame({'Bias_Wind-speed':[0]}),pd.DataFrame({'Bias_SR':[0]})]
for i in range(5):
    A = OW[i][OW[i]['Worker'].isin(vars()['TPA_'+str(i)]['Worker'])].filter(regex=task[i]).reset_index().drop('index',axis=1)
    B = GT_fit[i].loc[:5,].filter(regex=task[i])
    E = (abs(B - A) / B)
    for j in range(20):
        if j % 2 == 0:
#             E_all[i].index = j/2+1
            E_all[i].loc[j] = (np.sum(E[task[i]+str(j+1)]) + np.sum(E[task[i]+str(j+2)])) / (len(E.dropna()) * 2)
#             print(int(j/2+1),np.sum(E[task[i]+str(j+1)]) + np.sum(E[task[i]+str(j+2)]))
#             E_.append(task[i]+str(j+1),np.sum(E[task[i]+str(j+1)]) + np.sum(E[task[i]+str(j+2)]))
    D.append((abs(B - A) / B))

E_0 = E_all[0].reset_index().drop('index',axis=1)
E_0.index = E_0.index + 1
E_1 = E_all[1].reset_index().drop('index',axis=1)
E_1.index = E_1.index + 1
E_1.T
E_2 = E_all[2].reset_index().drop('index',axis=1)
E_2.index = E_2.index + 1
E_2.T
E_3 = E_all[3].reset_index().drop('index',axis=1)
E_3.index = E_3.index + 1
E_3.T
E_4 = E_all[4].reset_index().drop('index',axis=1)
E_4.index = E_4.index + 1
E = E_0.T.append(E_1.T.append(E_2.T.append(E_3.T.append(E_4.T))))
E.columns = ['t1','t2','t3','t4','t5','t6','t7','t8','t9','t10']
E.T

# 10. Mean method

In [ ]:
import time
start_time = time.time()
end_time = time.time()
def mean_truth_bias(gt_data,data,Task):
    bias_avg = 0
    data = data.filter(regex=Task)
    mean_truth_bias = pd.DataFrame({'Bias_'+Task:[0]})
    E = abs(gt_data - data.mean()) / gt_data
    for j in range(20):
        if j % 2 == 0:
            mean_truth_bias.loc[j] = (np.sum(E[Task+str(j+1)]) + np.sum(E[Task+str(j+2)])) / (len(E.dropna()) * 2)
    mean_truth_bias = mean_truth_bias.reset_index().drop('index',axis=1)
    mean_truth_bias.index = mean_truth_bias.index + 1
    mean_truth_bias = mean_truth_bias.T
    mean_truth_bias.columns = ['t1','t2','t3','t4','t5','t6','t7','t8','t9','t10']
    return mean_truth_bias
mean_execution_time = (end_time - start_time) * 1000
print("程序执行时间：", mean_execution_time, "毫秒")

mean_truth_bias(GT_fit[0],OW[0],'TEMP').append(mean_truth_bias(GT_fit[1],OW[1],'Humidity')
                                               .append(mean_truth_bias(GT_fit[2],OW[2],'Cloud')
                                                       .append(mean_truth_bias(GT_fit[3],OW[3],'Wind-speed')
                                                               .append(mean_truth_bias(GT_fit[4],OW[4],'SR'))))).T

In [ ]:
def mean_truth_accuracy(data,Task,subsize):
    mean_truth_accuracy = pd.DataFrame({'accuracy_'+Task:[0],'f1-score_'+Task:[0]})
    unknow_data = data.filter(regex=Task)
    bias = 2
    for i in range(20):
        if i % 2 == 0:
            unknow_data['mean_label'+str(int(i/2+1))]  = 0
            for j in range(len(data)):
                if (abs(unknow_data[Task+str(i+1)][j] - unknow_data[Task+str(i+1)].mean()) <= bias) and (abs(unknow_data[Task+str(i+2)][j] - unknow_data[Task+str(i+2)].mean()) <= bias):
                    unknow_data['mean_label'+str(int(i/2+1))][j] = 1
                else:
                    unknow_data['mean_label'+str(int(i/2+1))][j] = -1
    pred = unknow_data.filter(regex='mean')

    for t in range(10):
        y_pre = np.array(pred['mean_label'+str(t+1)]).reshape(-1).astype(int)
        mean_truth_accuracy.loc[t] = [accuracy_score(data['Label'],y_pre),f1_score(data['Label'],y_pre)]
    if Task == 'Humidity':Task = 'HUM'
    if Task == 'Cloud':Task = 'CLD'
    if Task == 'Wind-speed':Task = 'WSD'
    plot_roc(Task,data['Label'], y_pre,subsize)
    plt.savefig('3.svg',dpi=600,bbox_inches = 'tight')
    mean_truth_accuracy.index = mean_truth_accuracy.index + 1
    mean_truth_accuracy = mean_truth_accuracy.T
    mean_truth_accuracy.columns = ['t1','t2','t3','t4','t5','t6','t7','t8','t9','t10']
    return mean_truth_accuracy
plt.figure(figsize = (30,4))

mean_truth_accuracy(OW[0].loc[0:19],'TEMP',1).append(
    mean_truth_accuracy(OW[1].loc[0:19],'Humidity',1)).append(
    mean_truth_accuracy(OW[2].loc[0:19],'Cloud',1)).append(
    mean_truth_accuracy(OW[3].loc[0:19],'Wind-speed',1)).append(
    mean_truth_accuracy(OW[4].loc[0:19],'SR',1)).T

# 11. MV method

In [ ]:
import time
start_time = time.time()
end_time = time.time()

def mv_truth_bias(gt_data,data,Task):
    bias_avg = 0
    data = data.filter(regex=Task)
    mv_truth_bias = pd.DataFrame({'Bias_'+Task:[0]})
    E = abs(gt_data - data.round().mode()) / gt_data
    for j in range(20):
        if j % 2 == 0:
            mv_truth_bias.loc[j] = (np.sum(E[Task+str(j+1)]) + np.sum(E[Task+str(j+2)])) / (len(E.dropna()) * 2)
    mv_truth_bias = mv_truth_bias.reset_index().drop('index',axis=1)
    mv_truth_bias.index = mv_truth_bias.index + 1
    mv_truth_bias = mv_truth_bias.T
    mv_truth_bias.columns = ['t1','t2','t3','t4','t5','t6','t7','t8','t9','t10']
    return mv_truth_bias
mv_execution_time = (end_time - start_time) * 1000
print("程序执行时间：", mv_execution_time, "毫秒")

mv_truth_bias(GT_fit[0],OW[0],'TEMP').append(mv_truth_bias(GT_fit[1],OW[1],'Humidity')
                                               .append(mv_truth_bias(GT_fit[2],OW[2],'Cloud')
                                                       .append(mv_truth_bias(GT_fit[3],OW[3],'Wind-speed')
                                                               .append(mv_truth_bias(GT_fit[4],OW[4],'SR'))))).T

In [ ]:
def mv_truth_accuracy(data,Task,subsize):
    mv_truth_accuracy = pd.DataFrame({'accuracy_'+Task:[0],'f1-score_'+Task:[0]})
    unknow_data = data.filter(regex=Task)
    bias = 2
    for i in range(20):
        if i % 2 == 0:
            unknow_data['mv_label'+str(int(i/2+1))]  = 0
            for j in range(len(data)):
                if (abs(unknow_data[Task+str(i+1)][j] - unknow_data[Task+str(i+1)].round().mode()[0]) <= bias) and (
                    abs(unknow_data[Task+str(i+2)][j] - unknow_data[Task+str(i+2)].round().mode()[0]) <= bias):
                    unknow_data['mv_label'+str(int(i/2+1))][j] = 1
                else:
                    unknow_data['mv_label'+str(int(i/2+1))][j] = -1
    pred = unknow_data.filter(regex='mv')

    for t in range(10):
        y_pre = np.array(pred['mv_label'+str(t+1)]).reshape(-1).astype(int)
        mv_truth_accuracy.loc[t] = [accuracy_score(data['Label'],y_pre),f1_score(data['Label'],y_pre)]
    if Task == 'Humidity':Task = 'HUM'
    if Task == 'Cloud':Task = 'CLD'
    if Task == 'Wind-speed':Task = 'WSD'
    plot_roc(Task,data['Label'], y_pre,subsize,'upper left')# The last time-step
    plt.savefig('4.svg',dpi=600,bbox_inches = 'tight')
    mv_truth_accuracy.index = mv_truth_accuracy.index + 1
    mv_truth_accuracy = mv_truth_accuracy.T
    mv_truth_accuracy.columns = ['t1','t2','t3','t4','t5','t6','t7','t8','t9','t10']
    return mv_truth_accuracy
plt.figure(figsize = (30,4))

mv_truth_accuracy(OW[0].loc[0:19],'TEMP',1).append(mv_truth_accuracy(OW[1].loc[0:19],'Humidity',1)).append(
    mv_truth_accuracy(OW[2].loc[0:19],'Cloud',1)).append(mv_truth_accuracy(OW[3].loc[0:19],'Wind-speed',1)).append(
    mv_truth_accuracy(OW[4].loc[0:19],'SR',1)).T

# 12. Median method

In [ ]:
def median_truth_bias(gt_data,data,Task):
    bias_avg = 0
    data = data.filter(regex=Task)
    median_truth_bias = pd.DataFrame({'Bias_'+Task:[0]})
    E = abs(data - data.round().median()) / gt_data

    for j in range(20):
        if j % 2 == 0:
            median_truth_bias.loc[j] = (np.sum(E[Task+str(j+1)]) + np.sum(E[Task+str(j+2)])) / (len(E.dropna()) * 2)
    median_truth_bias = median_truth_bias.reset_index().drop('index',axis=1)
    median_truth_bias.index = median_truth_bias.index + 1
    median_truth_bias = median_truth_bias.T
    median_truth_bias.columns = ['t1','t2','t3','t4','t5','t6','t7','t8','t9','t10']
    return median_truth_bias

median_truth_bias(GT_fit[0],OW[0],'TEMP').append(median_truth_bias(GT_fit[1],OW[1],'Humidity')
                                               .append(median_truth_bias(GT_fit[2],OW[2],'Cloud')
                                                       .append(median_truth_bias(GT_fit[3],OW[3],'Wind-speed')
                                                               .append(median_truth_bias(GT_fit[4],OW[4],'SR')))))

In [ ]:
def median_truth_accuracy(data,Task,subsize):
    median_truth_accuracy = pd.DataFrame({'accuracy_'+Task:[0],'f1-score_'+Task:[0]})
    unknow_data = data.filter(regex=Task)
    bias = 2
    for i in range(20):
        if i % 2 == 0:
            unknow_data['mv_label'+str(int(i/2+1))]  = 0
            for j in range(len(data)):
                if (abs(unknow_data[Task+str(i+1)][j] - unknow_data[Task+str(i+1)].round().median()) <= bias) and (
                    abs(unknow_data[Task+str(i+2)][j] - unknow_data[Task+str(i+2)].round().median()) <= bias):
                    unknow_data['mv_label'+str(int(i/2+1))][j] = 1
                else:
                    unknow_data['mv_label'+str(int(i/2+1))][j] = -1
    pred = unknow_data.filter(regex='mv')

    for t in range(10):
        y_pre = np.array(pred['mv_label'+str(t+1)]).reshape(-1).astype(int)
        median_truth_accuracy.loc[t] = [accuracy_score(data['Label'],y_pre),f1_score(data['Label'],y_pre)]
    if Task == 'Humidity':Task = 'HUM'
    if Task == 'Cloud':Task = 'CLD'
    if Task == 'Wind-speed':Task = 'WSD'
    plot_roc(Task,data['Label'], y_pre,subsize,'upper left')# The last time-step
    plt.savefig('5.svg',dpi=600,bbox_inches = 'tight')
    median_truth_accuracy.index = median_truth_accuracy.index + 1
    median_truth_accuracy = median_truth_accuracy.T
    median_truth_accuracy.columns = ['t1','t2','t3','t4','t5','t6','t7','t8','t9','t10']
    return median_truth_accuracy
plt.figure(figsize = (30,4))

median_truth_accuracy(OW[0].loc[0:19],'TEMP',1).append(median_truth_accuracy(OW[1].loc[0:19],'Humidity',1)).append(
    median_truth_accuracy(OW[2].loc[0:19],'Cloud',1)).append(median_truth_accuracy(OW[3].loc[0:19],'Wind-speed',1)).append(
    median_truth_accuracy(OW[4].loc[0:19],'SR',1))

# 13. GD method

In [ ]:
import time
start_time = time.time()
end_time = time.time()

A = TPA_TEMP[TPA_TEMP['Label'] == 1]
B = OW_HUM[OW_HUM['Worker'].isin(A['Worker'])]
C = OW_CLD[OW_CLD['Worker'].isin(A['Worker'])]
D = OW_WSD[OW_WSD['Worker'].isin(A['Worker'])]
E = OW_SR[OW_SR['Worker'].isin(A['Worker'])]

def gd_truth_bias(gt_data,data,Task):
    bias_avg = 0
    data = data.filter(regex=Task)
    gd_truth_bias = pd.DataFrame({'Bias_'+Task:[0]})
    E = abs(gt_data - data) / gt_data
    for j in range(20):
        if j % 2 == 0:
            gd_truth_bias.loc[j] = (np.sum(E[Task+str(j+1)]) + np.sum(E[Task+str(j+2)])) / (len(E.dropna()) * 2)
    gd_truth_bias = gd_truth_bias.reset_index().drop('index',axis=1)
    gd_truth_bias.index = gd_truth_bias.index + 1
    gd_truth_bias = gd_truth_bias.T
    gd_truth_bias.columns = ['t1','t2','t3','t4','t5','t6','t7','t8','t9','t10']
    return gd_truth_bias
gd_execution_time = (end_time - start_time) * 1000
print("程序执行时间：", gd_execution_time, "毫秒")

gd_truth_bias(GT_fit[0],A,'TEMP').append(gd_truth_bias(GT_fit[1],B,'Humidity')
                                               .append(gd_truth_bias(GT_fit[2],C,'Cloud')
                                                       .append(gd_truth_bias(GT_fit[3],D,'Wind-speed')
                                                               .append(gd_truth_bias(GT_fit[4],E,'SR'))))).T

In [ ]:
reliable_TEMP = OW_TEMP[OW_TEMP['Worker'].isin(A['Worker'])]
reliable_TEMP['Label'] = 1
unreliable_TEMP = OW_TEMP[~OW_TEMP['Worker'].isin(A['Worker'])]
unreliable_TEMP['Label'] = -1
UNK_TEMP = reliable_TEMP.append(unreliable_TEMP).sort_index()

reliable_HUM = OW_HUM[OW_HUM['Worker'].isin(A['Worker'])]
reliable_HUM['Label'] = 1
unreliable_HUM = OW_HUM[~OW_HUM['Worker'].isin(A['Worker'])]
unreliable_HUM['Label'] = -1
UNK_HUM = reliable_HUM.append(unreliable_HUM).sort_index()

reliable_CLD = OW_CLD[OW_CLD['Worker'].isin(A['Worker'])]
reliable_CLD['Label'] = 1
unreliable_CLD = OW_CLD[~OW_CLD['Worker'].isin(A['Worker'])]
unreliable_CLD['Label'] = -1
UNK_CLD = reliable_CLD.append(unreliable_CLD).sort_index()

reliable_WSD = OW_WSD[OW_WSD['Worker'].isin(A['Worker'])]
reliable_WSD['Label'] = 1
unreliable_WSD = OW_WSD[~OW_WSD['Worker'].isin(A['Worker'])]
unreliable_WSD['Label'] = -1
UNK_WSD = reliable_WSD.append(unreliable_WSD).sort_index()

reliable_SR = OW_SR[OW_SR['Worker'].isin(A['Worker'])]
reliable_SR['Label'] = 1
unreliable_SR = OW_SR[~OW_SR['Worker'].isin(A['Worker'])]
unreliable_SR['Label'] = -1
UNK_SR = reliable_SR.append(unreliable_SR).sort_index()

plt.figure(figsize = (30,4))
plot_roc('TEMP',OW[0].loc[0:19,'Label'], UNK_TEMP[0:20]['Label'],1)
plot_roc('HUM',OW[1].loc[0:19,'Label'], UNK_HUM[0:20]['Label'],1)
plot_roc('CLD',OW[2].loc[0:19,'Label'], UNK_CLD[0:20]['Label'],1)
plot_roc('WSD',OW[3].loc[0:19,'Label'], UNK_WSD[0:20]['Label'],1)
plot_roc('SR',OW[4].loc[0:19,'Label'], UNK_SR[0:20]['Label'],1)
plt.savefig('6.svg',dpi=600,bbox_inches = 'tight')

In [ ]:
def gd_truth_accuracy(GT_data,UNK_data,Task):
    gd_truth_accuracy = pd.DataFrame({'accuracy_'+Task:[0],'f1-score_'+Task:[0]})
    unknow_data = UNK_data.filter(regex=Task)
    for t in range(10):
        gd_truth_accuracy.loc[t] = [accuracy_score(GT_data['Label'],UNK_data['Label']),f1_score(GT_data['Label'],UNK_data['Label'])]
    gd_truth_accuracy.index = gd_truth_accuracy.index + 1
    gd_truth_accuracy = gd_truth_accuracy.T
    gd_truth_accuracy.columns = ['t1','t2','t3','t4','t5','t6','t7','t8','t9','t10']
    return gd_truth_accuracy
gd_truth_accuracy(OW[0].loc[0:19],UNK_TEMP[0:20],'TEMP').append(gd_truth_accuracy(OW[1].loc[0:19],UNK_HUM[0:20],'Humidity')).append(
    gd_truth_accuracy(OW[2].loc[0:19],UNK_CLD[0:20],'Cloud')).append(gd_truth_accuracy(OW[3].loc[0:19],UNK_WSD[0:20],'Wind-speed')).append(
    gd_truth_accuracy(OW[4].loc[0:19],UNK_SR[0:20],'SR')).T

# Not datasets testing

In [ ]:
#mean temp
import numpy as np
gd_data = np.array([
[25.03,26.02,24.08,25.03,26.01],
[25.01,25.98,24.05,25.21,25.85],
[25.03,26.02,24.05,26.21,25.01],
[25.01,25.98,24.08,25.25,25.28],
[25.03,26.01,24.10,25.35,25.28]]
)

worker_report = np.array([
[25.03,26.02,24.08,25.03,26.01],
[27.88,28.32,26.7,28.83,30.21],
[25.03,26.02,24.05,25.25,26.82],
[27.98,28.98,26.78,28.85,30.26],
[27.21,28.21,26.54,28.68,30.26]
])

list_data = []
#mean
for i in range(5):
    list_data.append(abs(gd_data[:,i] - worker_report[:,i].mean()) / gd_data[:,i])
for i in range(5):
    print(sum(list_data[i]) )


In [ ]:
#MV temp
list_data2 = []
for i in range(5):
    list_data2.append(abs(gd_data[:,i] - np.argmax(np.bincount(list(worker_report[:,i])))) / gd_data[:,i])
for i in range(5):
    print(sum(list_data2[i]))


In [ ]:
#GD temp
list_data3 = []
b = np.isin(worker_report,gd_data)
print(b)

worker_report2 = np.array([
[25.03,26.02,24.08,25.03,26.01],
[25.01,25.98,24.05,25.21,25.85],
[25.03,26.02,24.05,26.21,26.82],
[25.01,25.98,24.08,25.25,25.28],
[25.03,26.01,24.10,25.35,25.28]]
)

for i in range(5):
    list_data3.append(abs(gd_data[:,i] - worker_report2[:,i])  / gd_data[:,i])
for i in range(5):
    print(sum(list_data3[i]))

In [ ]:
#mean water-level
import numpy as np
gd_data = np.array([
[150.23,160.01,150.28,150.55,160.23],
[150.13,159.23,151.30,149.89,161.08],
[152.56,160.33,151.02,151.22,160.33],
[150.33,159.53,151.73,150.78,159.45],
[153.45,161.33,149.23,150.01,159.75],
])


worker_report = np.array([
[155.23,165.01,155.28,155.55,165.23],
[170.22,175.31,170.25,174.31,175.85],
[171.13,160.33,151.02,151.22,160.33],
[165.13,169.23,171.30,169.89,171.08],
[150.23,159.53,151.02,149.89,159.45],
]
)


list_data = []
#mean
for i in range(5):
    list_data.append(abs(gd_data[:,i] - worker_report[:,i].mean()) / gd_data[:,i])
for i in range(5):
    print(sum(list_data[i]))


In [ ]:
#MV water-level
list_data_water = []
for i in range(5):
    list_data_water.append(abs(gd_data[:,i] - np.argmax(np.bincount(list(worker_report[:,i])))) / gd_data[:,i])

for i in range(5):
    print(sum(list_data_water[i]) )


In [ ]:
#GD water-level
list_data3 = []
worker_report2 = np.array([
[155.23,165.01,155.28,155.55,165.23],
[150.13,159.23,151.30,149.89,161.08],
[171.13,160.33,151.02,151.22,160.33],
[150.33,159.53,151.73,150.78,159.45],
[150.23,159.53,151.02,149.89,159.45],
]
)
for i in range(5):
    list_data3.append(abs(gd_data[:,i] - worker_report2[:,i]) / gd_data[:,i] )
for i in range(5):
    print(sum(list_data3[i] ))


In [ ]:
#TPAAD water-level
list_data3 = []
worker_report2 = np.array([
[150.23,160.01,150.28,150.55,160.23],
[150.13,159.23,151.30,149.89,161.08],
[171.13,160.33,151.02,151.22,160.33],
[150.33,159.53,151.73,150.78,159.45],
[150.23,159.53,151.02,149.89,159.45],
]
)
for i in range(5):
    list_data3.append(abs(gd_data[:,i] - worker_report2[:,i]) / gd_data[:,i] )
for i in range(5):
    print(sum(list_data3[i] ))
